In [1]:
import prep as p
import matplotlib.pyplot as plt

In [2]:
# Data Preparation and Cleaning
df = p.prepared_df()
df.head()



,ID,Age,Nationality,Overall,Potential,Club,International Reputation,Weak Foot,Skill Moves,Jersey Number,...,Inches,num_position,enum_d_work_rate,enum_o_work_rate,norm_wage,norm_value,total_stats,power_foot_num,Days_at_Club,enum_weights
0,158023,31,Argentina,94,94,FC Barcelona,5.0,4.0,4.0,10.0,...,67,3,1,1,565.0,110500.0,2244.0,1,5366,159
1,20801,33,Portugal,94,94,Juventus,5.0,4.0,5.0,7.0,...,74,3,0,2,405.0,77000.0,2265.0,0,244,183
2,190871,26,Brazil,92,93,Paris Saint-Germain,5.0,5.0,5.0,10.0,...,69,3,1,2,290.0,118500.0,2178.0,0,585,150
3,193080,27,Spain,91,93,Manchester United,4.0,3.0,1.0,1.0,...,76,0,1,1,260.0,72000.0,2220.0,0,2810,168
4,192985,27,Belgium,91,92,Manchester City,4.0,5.0,4.0,7.0,...,71,2,2,2,355.0,102000.0,2313.0,0,1289,154


In [ ]:
enum_df = p.enum_strings(df)
nation_idx = p.string_indices(df, 'Nationality')
club_idx = p.string_indices(df,'Club')
nation_idx
for i,v in enum_df.iterrows():
     print(nation_idx[nation_idx['index'] == v['num_nation']]['value'])


6    Argentina
Name: value, dtype: object
121    Portugal
Name: value, dtype: object
20    Brazil
Name: value, dtype: object
137    Spain
Name: value, dtype: object
13    Belgium
Name: value, dtype: object
13    Belgium
Name: value, dtype: object
35    Croatia
Name: value, dtype: object
155    Uruguay
Name: value, dtype: object
137    Spain
Name: value, dtype: object
134    Slovenia
Name: value, dtype: object
120    Poland
Name: value, dtype: object
59    Germany
Name: value, dtype: object
155    Uruguay
Name: value, dtype: object
137    Spain
Name: value, dtype: object
55    France
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
46    England
Name: value, dtype: object
55    France
Name: value, dtype: object
59    Germany
Name: value, dtype: object
13    Belgium
Name: value, dtype: object
137    Spain
Name: value, dtype: object
155    Uruguay
Name: value, dtype: object
59    Germany
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
77    Italy
N

137    Spain
Name: value, dtype: object
55    France
Name: value, dtype: object
55    France
Name: value, dtype: object
35    Croatia
Name: value, dtype: object
129    Scotland
Name: value, dtype: object
137    Spain
Name: value, dtype: object
77    Italy
Name: value, dtype: object
59    Germany
Name: value, dtype: object
77    Italy
Name: value, dtype: object
137    Spain
Name: value, dtype: object
121    Portugal
Name: value, dtype: object
77    Italy
Name: value, dtype: object
60    Ghana
Name: value, dtype: object
20    Brazil
Name: value, dtype: object
46    England
Name: value, dtype: object
77    Italy
Name: value, dtype: object
141    Sweden
Name: value, dtype: object
46    England
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
18    Bosnia Herzegovina
Name: value, dtype: object
59    Germany
Name: value, dtype: object
77    Italy
Name: value, dtype: object
46    England
Name: value, dtype: object
13    Belgium
Name: value, dtype: object
130    Senegal
Nam

Name: value, dtype: object
59    Germany
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
155    Uruguay
Name: value, dtype: object
71    Hungary
Name: value, dtype: object
137    Spain
Name: value, dtype: object
46    England
Name: value, dtype: object
46    England
Name: value, dtype: object
20    Brazil
Name: value, dtype: object
124    Republic of Ireland
Name: value, dtype: object
20    Brazil
Name: value, dtype: object
121    Portugal
Name: value, dtype: object
137    Spain
Name: value, dtype: object
137    Spain
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
55    France
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
134    Slovenia
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
77    Italy
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
46    England
Name: value, dtype: object
137    Spain
Name: value, dtype: object
13    Belgium
Name: value, dtype: object
133    Slovakia


Name: value, dtype: object
6    Argentina
Name: value, dtype: object
155    Uruguay
Name: value, dtype: object
31    Colombia
Name: value, dtype: object
55    France
Name: value, dtype: object
137    Spain
Name: value, dtype: object
55    France
Name: value, dtype: object
46    England
Name: value, dtype: object
106    Netherlands
Name: value, dtype: object
137    Spain
Name: value, dtype: object
20    Brazil
Name: value, dtype: object
39    Czech Republic
Name: value, dtype: object
121    Portugal
Name: value, dtype: object
59    Germany
Name: value, dtype: object
13    Belgium
Name: value, dtype: object
137    Spain
Name: value, dtype: object
46    England
Name: value, dtype: object
29    Chile
Name: value, dtype: object
141    Sweden
Name: value, dtype: object
137    Spain
Name: value, dtype: object
137    Spain
Name: value, dtype: object
137    Spain
Name: value, dtype: object
76    Israel
Name: value, dtype: object
121    Portugal
Name: value, dtype: object
55    France
Name: valu

Name: value, dtype: object
77    Italy
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
59    Germany
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
41    Denmark
Name: value, dtype: object
137    Spain
Name: value, dtype: object
13    Belgium
Name: value, dtype: object
55    France
Name: value, dtype: object
55    France
Name: value, dtype: object
20    Brazil
Name: value, dtype: object
131    Serbia
Name: value, dtype: object
106    Netherlands
Name: value, dtype: object
137    Spain
Name: value, dtype: object
80    Japan
Name: value, dtype: object
46    England
Name: value, dtype: object
59    Germany
Name: value, dtype: object
142    Switzerland
Name: value, dtype: object
99    Mexico
Name: value, dtype: object
55    France
Name: value, dtype: object
20    Brazil
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
51    FYR Macedonia
Name: value, dtype: object
55    France
Name: value, dtype: object
6    Argentina
Name: val

...

46    England
Name: value, dtype: object
20    Brazil
Name: value, dtype: object
59    Germany
Name: value, dtype: object
77    Italy
Name: value, dtype: object
59    Germany
Name: value, dtype: object
46    England
Name: value, dtype: object
125    Romania
Name: value, dtype: object
106    Netherlands
Name: value, dtype: object
78    Ivory Coast
Name: value, dtype: object
29    Chile
Name: value, dtype: object
103    Morocco
Name: value, dtype: object
55    France
Name: value, dtype: object
55    France
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
106    Netherlands
Name: value, dtype: object
157    Venezuela
Name: value, dtype: object
10    Azerbaijan
Name: value, dtype: object
154    United States
Name: value, dtype: object
55    France
Name: value, dtype: object
150    Turkey
Name: value, dtype: object
59    Germany
Name: value, dtype: object
46    England
Name: value, dtype: object
30    China PR
Name: value, dtype: object
31    Colombia
Name: value, dtype:

Name: value, dtype: object
113    Norway
Name: value, dtype: object
137    Spain
Name: value, dtype: object
20    Brazil
Name: value, dtype: object
20    Brazil
Name: value, dtype: object
154    United States
Name: value, dtype: object
154    United States
Name: value, dtype: object
104    Mozambique
Name: value, dtype: object
106    Netherlands
Name: value, dtype: object
124    Republic of Ireland
Name: value, dtype: object
24    Cameroon
Name: value, dtype: object
137    Spain
Name: value, dtype: object
80    Japan
Name: value, dtype: object
80    Japan
Name: value, dtype: object
131    Serbia
Name: value, dtype: object
112    Northern Ireland
Name: value, dtype: object
55    France
Name: value, dtype: object
106    Netherlands
Name: value, dtype: object
99    Mexico
Name: value, dtype: object
46    England
Name: value, dtype: object
46    England
Name: value, dtype: object
41    Denmark
Name: value, dtype: object
128    Saudi Arabia
Name: value, dtype: object
137    Spain
Name: valu

43    Ecuador
Name: value, dtype: object
55    France
Name: value, dtype: object
49    Estonia
Name: value, dtype: object
9    Austria
Name: value, dtype: object
31    Colombia
Name: value, dtype: object
29    Chile
Name: value, dtype: object
137    Spain
Name: value, dtype: object
9    Austria
Name: value, dtype: object
41    Denmark
Name: value, dtype: object
154    United States
Name: value, dtype: object
124    Republic of Ireland
Name: value, dtype: object
99    Mexico
Name: value, dtype: object
39    Czech Republic
Name: value, dtype: object
61    Greece
Name: value, dtype: object
105    Namibia
Name: value, dtype: object
99    Mexico
Name: value, dtype: object
141    Sweden
Name: value, dtype: object
77    Italy
Name: value, dtype: object
59    Germany
Name: value, dtype: object
106    Netherlands
Name: value, dtype: object
26    Cape Verde
Name: value, dtype: object
46    England
Name: value, dtype: object
55    France
Name: value, dtype: object
120    Poland
Name: value, dtype

154    United States
Name: value, dtype: object
99    Mexico
Name: value, dtype: object
128    Saudi Arabia
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
46    England
Name: value, dtype: object
46    England
Name: value, dtype: object
49    Estonia
Name: value, dtype: object
29    Chile
Name: value, dtype: object
157    Venezuela
Name: value, dtype: object
46    England
Name: value, dtype: object
59    Germany
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
41    Denmark
Name: value, dtype: object
124    Republic of Ireland
Name: value, dtype: object
31    Colombia
Name: value, dtype: object
46    England
Name: value, dtype: object
20    Brazil
Name: value, dtype: object
17    Bolivia
Name: value, dtype: object
59    Germany
Name: value, dtype: object
46    England
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
35    Croatia
Name: value, dtype: object
9    Austria
Name: value, dtype: object
124    Republic of Ireland
Na

Name: value, dtype: object
59    Germany
Name: value, dtype: object
137    Spain
Name: value, dtype: object
141    Sweden
Name: value, dtype: object
46    England
Name: value, dtype: object
130    Senegal
Name: value, dtype: object
80    Japan
Name: value, dtype: object
128    Saudi Arabia
Name: value, dtype: object
55    France
Name: value, dtype: object
8    Australia
Name: value, dtype: object
29    Chile
Name: value, dtype: object
130    Senegal
Name: value, dtype: object
137    Spain
Name: value, dtype: object
113    Norway
Name: value, dtype: object
6    Argentina
Name: value, dtype: object
30    China PR
Name: value, dtype: object
85    Korea Republic
Name: value, dtype: object
78    Ivory Coast
Name: value, dtype: object
142    Switzerland
Name: value, dtype: object
55    France
Name: value, dtype: object
54    Finland
Name: value, dtype: object
29    Chile
Name: value, dtype: object
72    Iceland
Name: value, dtype: object
80    Japan
Name: value, dtype: object
59    Germany
N

In [ ]:
plt.figure(figsize=(12,8))
goalkeepers = df[df.num_position == 0]
fielders = df[df.num_position != 0]
plt.scatter(goalkeepers['total_stats'], goalkeepers['norm_wage'])
plt.scatter(fielders['total_stats'], fielders['norm_wage'], alpha=.2)

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(df['norm_value'], df['norm_wage'])

In [ ]:
plt.figure(figsize=(12,8))
right = df[df.power_foot_num == 0]
left = df[df.power_foot_num == 1]
print(len(right))
print(len(left))
plt.scatter(right['total_stats'], right['norm_wage'],color='orange', alpha=.4, label='Right Foot')
plt.scatter(left['total_stats'], left['norm_wage'],color='green', alpha=.3, label='Left Foot')
plt.legend()

In [ ]:
countries = df.groupby('nunm')
countries.size()